<a href="https://colab.research.google.com/github/nhawrylyshyn/TAPAS-Experiments/blob/master/TAPAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/google-research/tapas/blob/master/notebooks/sqa_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The Google AI Language Team Authors

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
# Copyright 2019 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Running a Tapas fine-tuned checkpoint
---
This notebook shows how to load and make predictions with TAPAS model, which was introduced in the paper: [TAPAS: Weakly Supervised Table Parsing via Pre-training](https://arxiv.org/abs/2004.02349)

# Clone and install the repository


First, let's fetch the code from the github repository and install it

In [2]:
! git clone https://github.com/google-research/tapas.git

fatal: destination path 'tapas' already exists and is not an empty directory.


In [3]:
! pip install ./tapas

Processing ./tapas
  Created wheel for tapas: filename=tapas-0.0.1.dev0-cp36-none-any.whl size=178508 sha256=dce5f19a16b94ed79f961f00fcea1986a2377310ed48fbbb1eca3b1ad4f3c867
  Stored in directory: /tmp/pip-ephem-wheel-cache-aft0h79y/wheels/2f/0b/d5/7e7fd15d1eb9839bd9768eaa6af2e0446329927b0f0c351387
Successfully built tapas
  Found existing installation: tapas 0.0.1.dev0
    Uninstalling tapas-0.0.1.dev0:
      Successfully uninstalled tapas-0.0.1.dev0


# Fetch models fom Google Storage

Next we can get pretrained checkpoint from Google Storage. For the sake of speed, this is base sized model trained on [SQA](https://www.microsoft.com/en-us/download/details.aspx?id=54253). Note that best results in the paper were obtained with with a large model, with 24 layers instead of 12.

In [4]:
! gsutil cp gs://tapas_models/2020_04_21/tapas_sqa_base.zip . && unzip tapas_sqa_base.zip

Copying gs://tapas_models/2020_04_21/tapas_sqa_base.zip...
\ [1 files][  1.0 GiB/  1.0 GiB]   60.4 MiB/s                                   
Operation completed over 1 objects/1.0 GiB.                                      
Archive:  tapas_sqa_base.zip
replace tapas_sqa_base/model.ckpt.data-00000-of-00001? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: tapas_sqa_base/model.ckpt.data-00000-of-00001  
  inflating: tapas_sqa_base/model.ckpt.index  
  inflating: tapas_sqa_base/README.txt  
  inflating: tapas_sqa_base/vocab.txt  
  inflating: tapas_sqa_base/bert_config.json  
  inflating: tapas_sqa_base/model.ckpt.meta  


# Imports

In [7]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

In [8]:
from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils

# Load checkpoint for prediction

Here's the prediction code, which will create and `interaction_pb2.Interaction` protobuf object, which is the datastructure we use to store examples, and then call the prediction script.

In [9]:
os.makedirs('results/sqa/tf_examples', exist_ok=True)
os.makedirs('results/sqa/model', exist_ok=True)
with open('results/sqa/model/checkpoint', 'w') as f:
  f.write('model_checkpoint_path: "model.ckpt-0"')
for suffix in ['.data-00000-of-00001', '.index', '.meta']:
  shutil.copyfile(f'tapas_sqa_base/model.ckpt{suffix}', f'results/sqa/model/model.ckpt-0{suffix}')

In [10]:
max_seq_length = 512
vocab_file = "tapas_sqa_base/vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def predict(table_data, queries):
  table = [list(map(lambda s: s.strip(), row.split("|"))) 
           for row in table_data.split("\n") if row.strip()]
  examples = convert_interactions_to_examples([(table, queries)])
  write_tf_example("results/sqa/tf_examples/test.tfrecord", examples)
  write_tf_example("results/sqa/tf_examples/random-split-1-dev.tfrecord", [])
  
  ! python tapas/tapas/run_task_main.py \
    --task="SQA" \
    --output_dir="results" \
    --noloop_predict \
    --test_batch_size={len(queries)} \
    --tapas_verbosity="ERROR" \
    --compression_type= \
    --init_checkpoint="tapas_sqa_base/model.ckpt" \
    --bert_config_file="tapas_sqa_base/bert_config.json" \
    --mode="predict" 2> error


  results_path = "results/sqa/model/test_sequence.tsv"
  all_coordinates = []
  df = pd.DataFrame(table[1:], columns=table[0])
  display(IPython.display.HTML(df.to_html(index=False)))
  print()
  with open(results_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader: 
      coordinates = prediction_utils.parse_coordinates(row["answer_coordinates"])
      all_coordinates.append(coordinates)
      answers = ', '.join([table[row + 1][col] for row, col in coordinates])
      position = int(row['position'])
      print(">", queries[position])
      print(answers)
  return all_coordinates

# Predict - Use Fake Analyst Data

In [34]:
table = """Month   |Analyst        |Level|Escalations|Closures|False Positives
           January |Dawn Stubbs    |1    |18         |4       |1
           January |Daria Mcintyre |1    |49         |30      |11
           January |Mahdi Melia    |2    |34         |84      |5
           January |Myron Neville  |2    |52         |9       |93
           January |Colin Pope     |1    |19         |48      |86
           January |Missy Goulding |1    |81         |65      |27
           January |Braden Shields |2    |90         |97      |91
           January |Jayne Wheeler  |2    |59         |68      |100
           January |Edison Mohammed|1    |97         |46      |14
           January |Leslie Bryant  |3    |45         |29      |72
           February|Dawn Stubbs    |1    |40         |29      |16
           February|Daria Mcintyre |1    |27         |39      |37
           February|Mahdi Melia    |2    |64         |4       |16
           February|Myron Neville  |2    |23         |42      |91
           February|Colin Pope     |1    |27         |59      |91
           February|Missy Goulding |1    |84         |98      |63
           February|Braden Shields |2    |96         |54      |32
           February|Jayne Wheeler  |2    |24         |99      |66
           February|Edison Mohammed|1    |54         |18      |40
           February|Leslie Bryant  |3    |77         |25      |51"""

In [35]:
result = predict(table, ["is Dawn Stubbs an analyst",
      "in which month did she have the most escalations?"])

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Month,Analyst,Level,Escalations,Closures,False Positives
January,Dawn Stubbs,1,18,4,1
January,Daria Mcintyre,1,49,30,11
January,Mahdi Melia,2,34,84,5
January,Myron Neville,2,52,9,93
January,Colin Pope,1,19,48,86
January,Missy Goulding,1,81,65,27
January,Braden Shields,2,90,97,91
January,Jayne Wheeler,2,59,68,100
January,Edison Mohammed,1,97,46,14
January,Leslie Bryant,3,45,29,72



> is Dawn Stubbs an analyst
Dawn Stubbs, Dawn Stubbs
> in which month did she have the most escalations?
February


In [36]:
result = predict(table, ["which Analyst had the most Closures?"])

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Month,Analyst,Level,Escalations,Closures,False Positives
January,Dawn Stubbs,1,18,4,1
January,Daria Mcintyre,1,49,30,11
January,Mahdi Melia,2,34,84,5
January,Myron Neville,2,52,9,93
January,Colin Pope,1,19,48,86
January,Missy Goulding,1,81,65,27
January,Braden Shields,2,90,97,91
January,Jayne Wheeler,2,59,68,100
January,Edison Mohammed,1,97,46,14
January,Leslie Bryant,3,45,29,72



> which Analyst had the most Closures?
Jayne Wheeler


In [37]:
result = predict(table, ["which analyst had the most closures in January?"])  ## It gets this wrong and is limitation

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Month,Analyst,Level,Escalations,Closures,False Positives
January,Dawn Stubbs,1,18,4,1
January,Daria Mcintyre,1,49,30,11
January,Mahdi Melia,2,34,84,5
January,Myron Neville,2,52,9,93
January,Colin Pope,1,19,48,86
January,Missy Goulding,1,81,65,27
January,Braden Shields,2,90,97,91
January,Jayne Wheeler,2,59,68,100
January,Edison Mohammed,1,97,46,14
January,Leslie Bryant,3,45,29,72



> which analyst had the most closures in January?
Jayne Wheeler


In [38]:
table = """Month   |Analyst        |Level|Escalations|Closures|False Positives
           January |Dawn Stubbs    |1    |18         |4       |1
           January |Daria Mcintyre |1    |49         |30      |11
           January |Mahdi Melia    |2    |34         |84      |5
           January |Myron Neville  |2    |52         |9       |93
           January |Colin Pope     |1    |19         |48      |86
           January |Missy Goulding |1    |81         |65      |27
           January |Braden Shields |2    |90         |97      |91
           January |Jayne Wheeler  |2    |59         |68      |100
           January |Edison Mohammed|1    |97         |46      |14
           January |Leslie Bryant  |3    |45         |29      |72"""

In [40]:
result = predict(table, ["Analysts with at least 50 escalations?",
                         "Out of these, who had the most False Positives?"])  ## It gets this wrong i.e. limitation

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Month,Analyst,Level,Escalations,Closures,False Positives
January,Dawn Stubbs,1,18,4,1
January,Daria Mcintyre,1,49,30,11
January,Mahdi Melia,2,34,84,5
January,Myron Neville,2,52,9,93
January,Colin Pope,1,19,48,86
January,Missy Goulding,1,81,65,27
January,Braden Shields,2,90,97,91
January,Jayne Wheeler,2,59,68,100
January,Edison Mohammed,1,97,46,14
January,Leslie Bryant,3,45,29,72



> Analysts with at least 50 escalations?
Jayne Wheeler, Edison Mohammed, Braden Shields, Myron Neville, Missy Goulding
> Out of these, who had the most False Positives?
Jayne Wheeler


In [41]:
result = predict(table, ["How many Level 2 Analysts are there?"])  ## It is suppose to count this 

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Month,Analyst,Level,Escalations,Closures,False Positives
January,Dawn Stubbs,1,18,4,1
January,Daria Mcintyre,1,49,30,11
January,Mahdi Melia,2,34,84,5
January,Myron Neville,2,52,9,93
January,Colin Pope,1,19,48,86
January,Missy Goulding,1,81,65,27
January,Braden Shields,2,90,97,91
January,Jayne Wheeler,2,59,68,100
January,Edison Mohammed,1,97,46,14
January,Leslie Bryant,3,45,29,72



> How many Level 2 Analysts are there?
Braden Shields, Myron Neville, Mahdi Melia, Jayne Wheeler


In [42]:
result = predict(table, ["What level is Leslie Bryant?"])

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Month,Analyst,Level,Escalations,Closures,False Positives
January,Dawn Stubbs,1,18,4,1
January,Daria Mcintyre,1,49,30,11
January,Mahdi Melia,2,34,84,5
January,Myron Neville,2,52,9,93
January,Colin Pope,1,19,48,86
January,Missy Goulding,1,81,65,27
January,Braden Shields,2,90,97,91
January,Jayne Wheeler,2,59,68,100
January,Edison Mohammed,1,97,46,14
January,Leslie Bryant,3,45,29,72



> What level is Leslie Bryant?
3
